#  Анализ базы данных. SQL

#  Цель исследования:
Проанализироать базу данных и сделать запросы по заданию.

# Задачи:
- вывести первые строки таблиц
- написать запросы решаюшие задачи
- вывести результаты
- сделать вывод по каждому из них

In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [1]:
# устанавливаем параметры
db_config = {'user': 'praktiku****', # имя пользователя
'pwd': 'Sdf4$2****', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Открываю первые 5 строк таблиц

In [2]:
# таблица books
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# таблица authors
query = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# таблица ratings
query = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
# таблица reviews
query = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
# таблица publishers
query = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Задания

### Запрос №1. Посчитайте, сколько книг вышло после 1 января 2000 года;

In [7]:
query_1 =( 
'''SELECT
        COUNT(book_id) book_cnt
    FROM
        books
    WHERE
        publication_date>'2000-1-1'   ''')

con=engine.connect()

pd.io.sql.read_sql(sql=text(query_1), con = con)

,book_cnt
0,819


После 1янв 2000 года было выпущено 819 книг. 

Команда DISTINCT не изменила кол-ва - ID книг в таблице уникальные.

### Запрос №2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [8]:
query_2 = '''
            SELECT 
                DISTINCT tabl.book_id
                ,ROUND(tabl.avg,2) avg_rating
                , COUNT(review_id) OVER (Partition by tabl.book_id) reviews
            FROM(SELECT b.book_id, AVG(rating)
            FROM books as b JOIN ratings as r
            ON b.book_id=r.book_id
            GROUP BY 1) AS tabl LEFT JOIN reviews AS rw
            ON
            tabl.book_id=rw.book_id'''

con=engine.connect()

query_2=pd.io.sql.read_sql(sql=text(query_2), con = con)
query_2

,book_id,avg_rating,reviews
0,131,4.33,2
1,497,4.13,6
2,458,3.33,3
3,256,4.33,3
4,889,3.50,2
...,...,...,...
995,908,3.60,4
996,267,3.91,5
997,650,4.50,2
998,38,4.00,3


Стоит отметить, что для 6 книг отсутствуют обзоры. Понял это при слиянии таблиц.

### Запрос №3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так  исключу из анализа брошюры

In [9]:
query_3 =( 
'''SELECT
          p.publisher, count(b.book_id) books_cnt
    FROM
        books as b  Join publishers as p
        on
        b.publisher_id=p.publisher_id
    WHERE
        b.num_pages>50
    Group by 1
    Order by 2 desc
    Limit 1
        
        
       ''')

con=engine.connect()

pd.io.sql.read_sql(sql=text(query_3), con = con)

,publisher,books_cnt
0,Penguin Books,42


Penguin Books- издательство которое выпускает больше всех увесистых книг более 50 страниц.

### Запрос №4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
query_4 =( 
'''SELECT
        a.author
        , round(AVG(r.rating),2) avg_rating          
    FROM
        authors as a JOIN books as b
        ON a.author_id=b.author_id 
        JOIN ratings as r
        ON b.book_id=r.book_id
    WHERE
        b.book_id in 
        (SELECT distinct book_id          
        FROM ratings
        group by 1
        Having count(rating_id)>=50)
    GROUP BY 1
    ORDER BY 2 desc
    LIMIT 1 ''')

con=engine.connect()

pd.io.sql.read_sql(sql=text(query_4), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


J.K. Rowling самый в среднем высоко оцененный автор. Mary GrandPré иллюстратор работающий над книгами J.K. Rowling. В базе указанны вместе. 

### Запрос №5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
query_4 =( 
''' WITH tabl AS
    (SELECT COUNT(review_id) review_cnt, username
          
    FROM reviews AS rw  Join books AS b
         ON rw.book_id=b.book_id
    WHERE rw.username IN 
        (SELECT  username          
         FROM ratings
         GROUP BY 1
         HAVING COUNT(rating_id)>48) 
    GROUP BY 2)
        SELECT
            AVG(review_cnt) avg_review_cnt
        FROM
            tabl''')

con=engine.connect()

pd.io.sql.read_sql(sql=text(query_4), con = con)

,avg_review_cnt
0,24.0


Среднее кол-во обзоров у пользователей поставивших более 48 оценок - 24! Возможно, чем активнее пользователь оценивает книги, тем активнее он их читает и тем больше у него есть что сказать о книгах!